# Introdução

Esse caderno tem por objetivo a criação de um modelo básico de treinamento, utilizando o classificador [RandomForest](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html), que faça a predição de homologação de arquivamentos de procedimentos enviados à 1A.CAM do MPF.

Nesse modelo vamos passar a considerar os textos das íntegras.

A primeira passagem, serão criadas colunas com as palavras-chaves informadas pela área negocial que vão indicar a ausência ou presença do termo no texto da íntegra da peça de promoção do arquivamento.

Na segunda passagem, vamos passar a considerar unicamente o texto da íntegra.

**Nota**: os dados desse modelo foram recuperados de procedimentos que tiveram suas deliberações realizadas após o dia 02/07/2018, data em que a nova composição tomou posse na 1A.CAM.

# Contagem de termos mais frequentes

Vamos fazer uma análise dos textos das íntegras e tentar encontrar os termos mais frequentes.

O objetivo é:

- fazer uma confirmação das frequências dos termos com a lista de palavras-chaves que nos foi enviada;
- descobrir outras palavras que possam ser indicativos de homologação ou não.

Serão realizadas três análises:

- considerando todas as íntegras;
- considerando apenas as íntegras das peças que tiveram seu homologamento confirmado;
- considerando os não homologados;

Dessa análise, podemos criar colunas derivadas.

# Carga de dados e pré-processamento

Vamos fazer a carga dos dados.

Vamos ler os textos das íntegras, limpá-los e associar ao dataframe com os dados de homologação.

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

from sklearn import metrics

In [3]:
PATH = "../data/"

In [4]:
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/andrethiago/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/andrethiago/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
# carga dos textos

from os import listdir
from os.path import isfile, join
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = stopwords.words('portuguese')
stop_words.extend(['n', 'nº', 'n°', 'n.º', ',', '.', '!', '?', ';', ':', '...', 'º', '–', '/', '(', ')'])

def get_text(file):
    with open(file, encoding='utf-8') as f:
        data=' '.join(line.strip() for line in f)
        f.close()
    
    return data.strip()

def get_text_non_stop_words(text):
    filtered_text = ''
    for w in word_tokenize(text, language='portuguese'):
        if w not in stop_words:
            filtered_text += w
            filtered_text += ' '            
    return filtered_text.strip()   
    

folder_integras = f'{PATH}/integras-textos'

#onlyfiles = [f for f in listdir(folder_integras) if isfile(join(folder_integras, f))]

texts = {}
for file in listdir(folder_integras):
    if isfile(join(folder_integras, file)):
        texts[file.split('.')[0]] = get_text(join(folder_integras,file))
    else:
        print('is not file', file)

    
#texts['64280538']

In [6]:
# montando um DataFrame
df_original = pd.read_json(f'{PATH}/1A.CAM.homologacao-arquivamento.json')
df_work = df_original.copy()
df_work.dropna(subset=['identificadorPecaPromocao'], inplace=True)
df_work.reset_index(drop=True, inplace=True)
cols = ['id', 'homologado']
df_work.drop(inplace=True, columns=[col for col in df_work.columns if col not in cols])
df_work['peca_promocao'] = None
df_work.sample(5)

,homologado,id,peca_promocao
4972,1,63581056,None
1081,1,84000484,None
4759,1,69279080,None
5386,1,65122401,None
1276,1,66738614,None


In [7]:
# Associando o texto...
for key, text in zip(texts.keys(), texts.values()):
    if key != '':
        df_work.loc[df_work.id == int(key), 'peca_promocao'] = text
    
df_work.sample(5)

,homologado,id,peca_promocao
2594,1,78580922,MINISTÉRIO PÚBLICO FEDERAL PROCURADORIA DA REP...
5132,1,67296888,
4541,1,69926981,MINISTÉRIO PÚBLICO FEDERAL Procuradoria da Re...
4031,1,70438817,Ofício MPF/PRPE/AT nº 171 /2003 Notícia de Fa...
2670,1,73584328,


In [7]:
#Carga das palavras chaves

palavras_chaves = pd.read_csv(f'{PATH}/1A.CAM.palavras-chaves.txt')
palavras_chaves

,palavras-chaves
0,AUSÊNCIA DE IRREGULARIDADES
1,IRREGULARIDADE SANADA
2,NÃO COMPROVAÇÃO
3,INEXISTÊNCIA
4,SOLUCIONADA
5,EXAURIMENTO
6,EXAURIDA
7,PERDA DE OBJETO
8,IRREGULARIDADE NÃO COMPROVADA
9,INEXISTÊNCIA DE INDÍCIOS DE IRREGULARIDADE


In [17]:
# Ver se dá pra usar o sklearn aqui...
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(ngram_range=(1,6))
bag = count.fit_transform(list(texts.values()))

In [33]:
filtered_text = [w for w in word_tokens if not w in stop]

In [23]:
from heapq import nlargest

top_ten = nlargest(10, count.vocabulary_, key=count.vocabulary_.get)

print(top_ten)

['üá qep 7ô w640 tej 61', 'üá qep 7ô w640 tej', 'üá qep 7ô w640', 'üá qep 7ô', 'üá qep', 'üá', 'üsuc ogrto oa rei jc1 iii', 'üsuc ogrto oa rei jc1', 'üsuc ogrto oa rei', 'üsuc ogrto oa']
